In [6]:
import os
import random
from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

from src.model import Model
from src import utils
from src.pytorch_utils import utils as ptu
from config import cfg
import src.data_loader as dl

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [15]:
model = Model(backbone=cfg.backbone, num_classes=cfg.num_classes, feature_extraction_process=True)
model.to(device)

Model(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0):

In [9]:
train_loader, train_eval_loader = dl.train_eval_loaders_cifar10(data_dir=cfg.data_dir, batch_size=60000)
test_loader = dl.test_loader(data_dir=cfg.data_dir, batch_size=60000)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [10]:
normalize = transforms.Normalize(utils.cifar10_mean, utils.cifar10_std)
transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

train_dataset = datasets.CIFAR10(
    root=cfg.data_dir, train=True,
    download=True, transform=transform,
)
print(len(train_dataset))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64,
    num_workers=cfg.num_workers
)


test_dataset = datasets.CIFAR10(
    root=cfg.data_dir, train=False,
    download=True, transform=transform,
)
print(len(test_dataset))

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64,
    num_workers=cfg.num_workers
)

Files already downloaded and verified
50000
Files already downloaded and verified
10000


In [11]:
train_data=None
train_targets=None
for i, batch in enumerate(tqdm(train_loader)):
    imgs, targets = batch
    imgs, targets = imgs.to(device), targets.to(device)
    out = model(imgs)
    if i == 0:
        train_data = out
        train_targets = targets
    else:
        train_data = torch.cat((train_data, out), 0)
        train_targets = torch.cat((train_targets, targets), 0)
print(train_data.shape, train_targets.shape)

test_data=None
test_targets=None
for i, batch in enumerate(tqdm(test_loader)):
    imgs, targets = batch
    imgs, targets = imgs.to(device), targets.to(device)
    out = model(imgs)
    if i == 0:
        test_data = out
        test_targets = targets
    else:
        test_data = torch.cat((test_data, out), 0)
        test_targets = torch.cat((test_targets, targets), 0)
print(test_data.shape, test_data.shape)

  0%|          | 0/157 [00:00<?, ?it/s]

torch.Size([50000, 2048]) torch.Size([50000])


100%|██████████| 157/157 [00:03<00:00, 40.37it/s]

torch.Size([10000, 2048]) torch.Size([10000, 2048])


In [12]:
train_data = train_data.detach().to('cpu')
test_data = test_data.detach().to('cpu')

train_targets = train_targets.detach().to('cpu')
test_targets = test_targets.detach().to('cpu')

In [13]:
torch.save(train_data, cfg.data_dir + '/exctracted_data/train.pt')
torch.save(test_data, cfg.data_dir + '/exctracted_data/test.pt')

torch.save(train_targets, cfg.data_dir + '/exctracted_data/train_targets.pt')
torch.save(test_targets, cfg.data_dir + '/exctracted_data/test_targets.pt')

In [14]:
torch.load(cfg.data_dir + '/exctracted_data/train.pt')

tensor([[0.1265, 0.8046, 0.0000,  ..., 0.0000, 0.3309, 0.0000],
        [0.1117, 0.1340, 0.0000,  ..., 0.0000, 0.0000, 0.1305],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4759, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.7541],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1348, 0.7166],
        [0.0000, 1.8082, 0.6041,  ..., 0.7243, 0.3153, 0.3343]])